# Typical use case for physical parameters

This example notebook shows a run on the COSMOS2020 (Weaver et al. 2022) data set in order to estimate the physical parameters.

In this notebook we follow what has been done for photo-z. We will be looking at the same COSMOS data set but only use the ugrizy bands. The main difference is the use of BC03 templates to compute physical parameters, and we set the redshift to its spectrosocpic value.

The notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Typical_use_case_physicalParameters.ipynb).

In [ ]:
import lephare as lp
from astropy.table import Table
import numpy as np
import os
from matplotlib import pylab as plt
import time

%matplotlib inline
%load_ext wurlitzer

## Update the config
We will start with the COSMOS configuration as a basis. We will update the various keywords needed for this example. We use the default which is shipped with lephare. You could also download the example text file config from [here](https://github.com/lephare-photoz/lephare-data/blob/main/examples/COSMOS.para) or write it completely from scratch.

In [ ]:
config = lp.default_cosmos_config.copy()

# You could also load this from a local text file:
# config=lp.keymap_to_string_dict(lp.read_config("your_own.para"))
# An example can be downloaded with curl
# !curl -s -o ./COSMOS.para https://raw.githubusercontent.com/lephare-photoz/lephare-data/refs/heads/main/examples/COSMOS.para
# config = lp.read_config("./COSMOS.para")

config.update(
    {
        # For a quick demonstration we use a very sparse redshift grid. DO NOT USE FOR SCIENCE!
        # Comment out the following line to improve results.
        "Z_STEP": "0.1,0.,3.",
        # SED
        # In order to get the physical parameters you need to use
        # Composite Stellar Population synthesis models. Here Bruzual & Charlot (2003).
        # This can be done only for galaxies.
        "GAL_SED": "$LEPHAREDIR/sed/GAL/BC03_CHAB/BC03COMB_MOD.list",
        # Limit the number of ages
        "SEL_AGE": "$LEPHAREDIR/sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",
        "MOD_EXTINC": "0,12,0,12",
        "EXTINC_LAW": "SB_calzetti.dat,SMC_prevot.dat",
        "EM_LINES": "PHYS",
        "EM_DISPERSION": "1.",
        # FILTERS
        # A reduced list of filters:
        "FILTER_LIST": "cosmos/u_new.pb,hsc/gHSC.pb,hsc/rHSC.pb,hsc/iHSC.pb,hsc/zHSC.pb,hsc/yHSC.pb",
        "FILTER_CALIB": "0",
        "FILTER_FILE": "filter_test",
        # FIT
        # We set the redshift to the spec-z value
        "ZFIX": "YES",
        "ERR_SCALE": "0.02",
        "ERR_FACTOR": "1.5",
        "SPEC_OUT": "NO",  # We would like to see the output
        "VERBOSE": "NO",
    }
)

## Download the missing data
One does not need to use this functionality if already cloned the full auxiliary data.



In [ ]:
lp.data_retrieval.get_auxiliary_data(
    keymap=config,
    # The additional extinction laws for galaxies are not in the principle config
    # so we must add them to be downloaded:
    additional_files=[
        # We also want the example cosmos catalogue to experiment with
        "examples/COSMOS.in",
        "ext/SMC_prevot.dat",
        "ext/SB_calzetti.dat",
        "sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",
    ],
)

## Run prepare

These are the key preparatory stages that calculate the filters in the LePHARE format, calculate the library of SEDs and finally calculate the library of magnitudes for all the models. The prepare method runs *filter*, *sedtolib*, and *mag_gal* that would be run independently at the command line. These are all explained in detail in the [documentation](https://lephare.readthedocs.io/en/latest/original.html#detailed-lephare-user-manual).

In [ ]:
lp.prepare(
    config,
)

### Creating the input table

We need to make an astropy table as input. This can be done using the standard column order:
id, flux0, err0, flux1, err1,..., context, zspec, arbitrary_string. A simple example table with two filters might look like this:
|  id | flux_filt1  |  fluxerr_filt1 |  flux_filt2  |  fluxerr_filt2 | context | zspec | string_data |
|---|---|---|---|---|---|---|---|
|  0 | 1.e-31  | 1.e-32  | 1.e-31  | 2.e-32  | 3 | NaN | "This is just a note" |
|  1 | 2.e-31  |  1.e-32 | 1.e-31  | 2.e-32  |3 | 1. | "This has a specz" |
|  2 | 2.e-31 | 1.e-32  | 2.e-31  | 2.e-32  | 2 | NaN| "This context only uses the second filter" |

The context detemermines which bands are used but can be -99 or a numpy.nan. We do not need to have units on the flux columns but LePHARE assumes they are in erg /s /cm**2 / Hz if we are using fluxes. The number of columns must be two times the number of filters plus the four additional columns.

This input table **must use** the standard column ordering to determine column meaning. This odering depends on the filter order in the config FILTER_LIST value. 

In [ ]:
# Load the full cosmos example we downloaded at the start
cosmos_full = Table.read(f"{lp.LEPHAREDIR}/examples/COSMOS.in", format="ascii")
# Lets just look at the first 1000 specz between 0 and 3 to be fast and have a small sample to test
specz_colname = cosmos_full.colnames[-2]
mask = cosmos_full[specz_colname] > 0

mask &= cosmos_full[specz_colname] < 3
cosmos_full = cosmos_full[mask][:1000]

In [ ]:
input_table = Table()
# The id is in the first column
input_table["id"] = cosmos_full[cosmos_full.colnames[0]]
# Loop over the filters we want to keep to get the number of the filter, n, and the name, b,
filter_names = config["FILTER_LIST"].split(",")
for n, filter_name in enumerate(filter_names):
    # The ugrizy fluxes and errors are in cols 3 to 14
    f_col = cosmos_full.colnames[2 * n + 3]
    ferr_col = cosmos_full.colnames[2 * n + 4]
    # By default lephare uses column order so names are irrelevant
    input_table[f"f_{filter_name}"] = cosmos_full[f_col]
    input_table[f"ferr_{filter_name}"] = cosmos_full[ferr_col]
# The context is a binary flag. Here we set it to use all filters.
input_table["context"] = np.sum(2 ** np.arange(len(filter_names)))
input_table["zspec"] = cosmos_full[specz_colname]
input_table["string_data"] = "arbitrary_info"

In [ ]:
# Look at the first 5 lines of the input table
input_table[:5]

## Run process

Finally we run the main fitting process which is equivalent to *zphota* when using the command line. We also need to update some of the config values to make them consistent with the number of filters.

In [ ]:
# Compute physical parameters
output, _ = lp.process(config, input_table)

In [ ]:
# the output is an astropy tabel that can be manipulated in the standard ways.
output[:5]

Next we can perform some simple plots to check the output

In [ ]:
logmass = output["MASS_MED"]
logSFR = output["SFR_MED"]
z = output["Z_BEST"]
cond = (z > 0.5) & (z < 1) & (logmass > 0)
plt.hist(logmass[cond], bins=20)
plt.xlabel("log(stellar mass)")

In [ ]:
plt.scatter(logmass[cond], logSFR[cond], s=2.0)
plt.xlabel("log(stellar mass)")
plt.ylabel("log(SFR)")

You can use some plotting utilities provided with the code. Some examples are provided below. 
sel_filt is the index of the filter used to select objects by observed magnitude (starting at 0).
pos_filt provides the index (starting at 0) of the filters corresponding to u, g, r, z, J, and Ks bands.
You can use some plotting utilities provided with the code. Some examples are provided below.
If you want to create all the plots and store them in a pdf file, you can use:
utils.save_phys_plots_pdf(filename="all_phys.pdf")

In [ ]:
utils = lp.PlotUtils(
    output,
    sel_filt=3,
    pos_filt=[0, 1, 2, 4, 5, 5],
    range_z=[0, 0.5, 1, 1.5, 3],
    range_mag=[19, 20.5, 21.5, 22.5, 25],
)

In [ ]:
utils.dist_chi2()

In [ ]:
utils.dist_mass()

In [ ]:
utils.mass_sfr()